### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [12]:
# Get dataframe from csv data
purchase_df = pd.DataFrame(purchase_data)
# purchase_df.head()

In [3]:
# Find and count all unique players and store in dictionary
all_players = purchase_df['SN'].unique()
player_d = {'Total Players':len(all_players)}

# Print dictionary as dataframe
pd.DataFrame([player_d])

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
# make new dataframe with only unique items
uitems_df = purchase_df.drop_duplicates(subset='Item ID', keep="last")

# Get number of items and average price
n_uitems = len(uitems_df['Item ID'])
avg_price = round(uitems_df['Price'].mean() + 0.005,2) # add half a cent to ensure we are always round up to the nearest cent
                                              # baised off of the real average
# get total purchase info    
n_purchases = len(purchase_df['Item ID'])
total_rev = purchase_df['Price'].sum()

# store into dictionary
ptotal_d = {
    'Number of Unique Items':n_uitems,
    'Average Price':f"${avg_price}",
    'Number of Purchases':n_purchases,
    'Total Revenue':f"${total_rev}"
}

# make a dataframe of the dictionary
ptotal_df = pd.DataFrame([ptotal_d])
ptotal_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
# Get dataframe with unique customer names and number of unique customers
customer_df = purchase_df.drop_duplicates(subset='SN', keep="last")
n_ucustomer = len(customer_df['SN'])

# Get overall gender info from unique customer dataframe
gender = customer_df['Gender'].value_counts()

# Add info to dictionary
gender_d = {
    'Total Count':{'Male':gender[0],'Female':gender[1],'Other / Non-Disclosed':gender[2]},
    'Percentage of Players':{'Male':f"{gender[0]/n_ucustomer*100:.2f}%",'Female':f"{gender[1]/n_ucustomer*100:.2f}%",
                             'Other / Non-Disclosed':f"{gender[2]/n_ucustomer*100:.2f}%"}
}
# convert dictionary to dataframe
gender_df = pd.DataFrame(gender_d)
gender_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
# get dataframes based off of gender
female_df = purchase_df.loc[purchase_df['Gender'] == 'Female',:]
male_df = purchase_df.loc[purchase_df['Gender'] == 'Male',:]
ond_df = purchase_df.loc[purchase_df['Gender'] == 'Other / Non-Disclosed',:]

# define a useful DataFrame list
df_list = [female_df,male_df,ond_df]

# make a function to get avg total purchase per person per gender dataframe
def gavg(DF):
    temp_avg = DF[['SN','Price']].groupby(['SN']).sum()
    return round(temp_avg['Price'].mean(),2)

# Start a dictionary for storing the info
g_analysis_d = {
    'Gender':['Female','Male','Other / Non-Disclosed'],
    'Purchase Count':[],
    'Average Purchase Price':[],
    'Total Purchase Value':[],
    'Avg Total Purchase per Person':[]
}

# calculate and add info for each Dataframe in the df_list into the analysis dictionary
for dataframe in df_list:
    g_analysis_d['Purchase Count'] += [len(dataframe['Purchase ID'])]
    g_analysis_d['Average Purchase Price'] += [f"${dataframe['Price'].mean():.2f}"]
    g_analysis_d['Total Purchase Value'] += [f"${dataframe['Price'].sum()}"]
    g_analysis_d['Avg Total Purchase per Person'] += [f"${gavg(dataframe)}"]

# Convert dict to df and format index
g_analysis_df = pd.DataFrame(g_analysis_d)
g_analysis_df = g_analysis_df.set_index('Gender')
g_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
# Create the bin for ages ranges
bins = [0,9,14,19,24,29,34,39,10000]
# Create the label for the age ranges
age_names = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

age_range = []
age_range[:] = pd.cut(customer_df["Age"],bins,labels=age_names,include_lowest=True)[:]
# Added age range this way due to error message, still getting it even after error.
customer_df.loc[:,'Age Range'] = age_range[:]

# Create a groupby with just age information 
age_g = customer_df[['Age','Age Range']].groupby(['Age Range'])
# Get appropriate count of each age range
final_age_df = pd.DataFrame(age_g['Age'].count())
final_age_df = final_age_df.loc[age_names,:] # reformate with age_names so that <10 is the first row
# Get total number of players
total_players = final_age_df.sum()
# Solve for percentage of players in each age range
p_percent = [x/total_players for x in final_age_df['Age']]
# Add percentages to the df and rename 'Age' column
final_age_df.loc[:,'Percentage of Players'] = p_percent[:]
final_age_df = final_age_df.rename(columns={'Age':'Total Count'})

# Show df with correct formating
## idea from stackoverflow for showing with correct format, but keeping actuall data as float
## https://stackoverflow.com/questions/23981601/format-certain-floating-dataframe-columns-into-percentage-in-pandas
final_age_df.style.format({'Percentage of Players':'{:,.2%}'.format})

C:\Users\jgeis\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\jgeis\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
# Use previous bin and age_names to create age range column again
p_age = []
p_age[:] = pd.cut(purchase_df["Age"],bins,labels=age_names,include_lowest=True)[:]
p_age_df = purchase_df.loc[:,:]
p_age_df['Age Range'] = p_age

# Get 'Purchase Count', 'Average Purchase Price', and 'Total Purchase Value' using groupby functions
## Get age range groupby
p_age_g = p_age_df[['Age Range','Price']].groupby(['Age Range'])
## Add summary dataframe
age_summary_df = pd.DataFrame(index=age_names)
## Add the groupby columns to the summary df
age_summary_df['Purchase Count'] = p_age_g.count()
age_summary_df['Average Purchase Price'] = p_age_g.mean()
age_summary_df['Total Purchase Value'] = p_age_g.sum()


# Get avg total purchase price per person in age range
## Groupby 'Age Range' and 'SN'
p_agename_g = p_age_df[['Age Range','Price','SN']].groupby(['Age Range','SN'])
## Get total amount spent per person per age range and change index back to default
p_agesum_df = p_agename_g.sum().reset_index()
## Finish by doing groupby on just 'Age Range' this time and store average in the df
p_agemean_g = p_agesum_df[['Age Range','Price']].groupby(['Age Range'])
age_summary_df['Avg Total Purchase per Person'] = p_agemean_g.mean()


# Handle all final formating chages and output result
age_summary_df.index.names = ['Age Range']
age_summary_df.style.format({
                            'Average Purchase Price':'${:,.2f}'.format,
                            'Total Purchase Value':'${:,.2f}'.format,
                            'Avg Total Purchase per Person':'${:,.2f}'.format
                            })


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# Get a groupby using just 'SN' name and the the amount spent on the transaction
tops_g = purchase_df[['SN','Price']].groupby(['SN'])
# Find the top 5 spenders after summing up all of their purchases
top5 = tops_g.sum().nlargest(5,'Price')
top5_spenders = list(top5.index)

# Create empty summary df with the index being the top 5 spenders
top_spenders_df = pd.DataFrame(index=top5_spenders)

# Using top5 spenders, find all purchases made by them
top5_df = purchase_df[['SN','Price']].loc[purchase_df['SN'].isin(top5_spenders),:]
# groupby 'SN' on the dataframe only including purchases of the 5 top spenders
top5_g = top5_df.groupby('SN')

# Add everything to summary df
top_spenders_df['Purchase Count'] = top5_g.count()
top_spenders_df['Average Purchase Price'] = top5_g.mean()
top_spenders_df['Total Purchase Value'] = top5['Price']

# Display the formated version
top_spenders_df.index.names = ['SN']
top_spenders_df.style.format({
                            'Average Purchase Price':'${:,.2f}'.format,
                            'Total Purchase Value':'${:,.2f}'.format
                            })

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
# Get a groupby using Item ID and Name
tops_g = purchase_df[['Item ID','Item Name','Price']].groupby(['Item ID','Item Name'])
# Find the top 6 most popular items(6 as 4 items sold 9 times) after counting their purchases
top6 = tops_g.count().nlargest(6,'Price')

# Create empty dataframe using index of 6 most popular items
most_pop_df = pd.DataFrame(index=top6.index)

# Store item IDs of the 6 items in list for easy filtering
items = [top6.index[x][0] for x in range(0,6)]
# Create df of all purchases of the 6 most popular items
top6_df = purchase_df[['Item ID','Item Name','Price']].loc[purchase_df['Item ID'].isin(items),:]

# Groupby on the top6 dataframe to get info filtered by item
top6_g = top6_df.groupby(['Item ID','Item Name'])

# Store most popular item purchase info in dataframe
most_pop_df['Purchase Count'] = top6_g.count()
most_pop_df['Item Price'] = top6_g.mean()
most_pop_df['Total Purchase Value'] = top6_g.sum()

# Display the formated version
most_pop_df.style.format({
                            'Item Price':'${:,.2f}'.format,
                            'Total Purchase Value':'${:,.2f}'.format
                            })

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
132,Persuasion,9,$3.22,$28.99
145,Fiery Glass Crusader,9,$4.58,$41.22


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [11]:
### The only difference with this cell from the most popular items is that top5 is found with sum()
### instead of count()

# Get a groupby using Item ID and Name
tops_g = purchase_df[['Item ID','Item Name','Price']].groupby(['Item ID','Item Name'])
# Find the top 5 spenders after summing up all of their purchases
top5 = tops_g.sum().nlargest(5,'Price')

# Create empty dataframe using index of top 5 most profitable items
most_pro_df = pd.DataFrame(index=top5.index)

# Store item IDs of the 5 items in list for easy filtering
items = [top5.index[x][0] for x in range(0,5)]
# Create df of all purchases of the 5 most profitable items
top5_df = purchase_df[['Item ID','Item Name','Price']].loc[purchase_df['Item ID'].isin(items),:]

# Groupby on the top5 df to get info filtered by item
top5_g = top5_df.groupby(['Item ID','Item Name'])

# Store most profitable item purchase info in dataframe
most_pro_df['Purchase Count'] = top5_g.count()
most_pro_df['Item Price'] = top5_g.mean()
most_pro_df['Total Purchase Value'] = top5_g.sum()

# Display the formated version
most_pro_df.style.format({
                            'Item Price':'${:,.2f}'.format,
                            'Total Purchase Value':'${:,.2f}'.format
                            })

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
